In [1]:
!pip install pyDOE
!pip install cma


  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=dcb2ca9074eb8178415ede922dbd2b96f3757376b4e8b0356e3c624e7776b23c
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 2.8MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import PolynomialFeatures
import bbobbenchmarks as bn
import sklearn

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F15 '''
def F15(X):
    f = bn.F15()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data

''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))
      

''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 0.285 ,random_state=0 , l1_ratio=0.9539, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 69.69 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_15_2000Samples.csv"
train_y = pd.read_csv(path).iloc[:,-1]
test_y = pd.read_csv('test_15_400Samples.csv').iloc[:,-1]
path = "train_latent_100D.csv"
train = pd.read_csv(path, index_col = 0)
test = pd.read_csv('test_latent_100D.csv', index_col = 0)
train ['Y'] = train_y
test ['Y'] = test_y
del train_y
del test_y
true = np.array(test['Y'])

## Surrogate Models

In [5]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,51):
        Cols.append(Columns[j]+'_Z'+str(i))

opt = cma.CMAOptions()
opt.set ("seed" , 0)
opt.set ("maxfevals" , 50000)

n_obs , dim =  30, 50
G = DOE(n_obs, dim)
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([30,150])
for i in range(G[1].shape[0]):
    print ('Run : '+ str(i))
    min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 0.2689433305566933 , options=opt) [0]
    min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 0.2689433305566933 , options=opt) [0]
    min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 0.2689433305566933 , options=opt) [0]
    X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimension 50 (seed=940304, Mon Apr 13 14:47:45 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     15 1.166707005013709e+04 1.0e+00 2.57e-01  3e-01  3e-01 0:00.0
    2     30 1.166701077252071e+04 1.1e+00 2.50e-01  2e-01  3e-01 0:00.0
    3     45 1.166700254751244e+04 1.1e+00 2.47e-01  2e-01  2e-01 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [11667.56878808] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [11667.05083555] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1500 -1.027331448082513e+04 1.5e+00 3.69e-02  3e-02  4e-02 0:02.0
  200   3000 -1.136779824171086e+04 1.6e+00 2.35e-03  2e-03  2e-03 0:03.9
  300   4500 -1.137231893459754e+04 1.6e+00 1.16e-04  1e-04  1e-04 0:05.8
  400   6000 -1.137232773808747e+04 1.5e+00 6.36e-06  5e-06  6e-06 0:07.7
  500   7500 -1.137232776772518e+04 1.6e+00 4.27e-07  3e-07  4e-07 0:09.6
  600   9000 -1.137232776784984e+04 1.6e+00 2.78e-08  2e-08  2e-08 0:11.5
  700  10500 -1.137232776785040e+04 1.7e+00 3.54e-09  3e-09  3e-09 0:13.5
  800  12000 -1.137232776785040e+04 1.9e+00 1.24e-09  9e-10  1e-09 0:15.4
  900  13500 -1.137232776785041e+04 2.2e+00 6.83e-10  5e-10  6e-10 0:17.3
 1000  15000 -1.137232776785041e+04 2.3e+00 5.71e-10  4e-10  5e-10 0:19.2
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  16500 -1.137232776785041e+04 2.6e+00 7.88e-10  6e-10  7e-10 0:21.2
 1200  18000 -1.137232776785041e+04 2.7e+00 6.47e-10  4e-10  5e-10 0:23.1
 1300  19500 -1.137232776785042e+04 2.9e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [10895.81357622] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=10895.813576, sigma=2.55e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=10895.813576, sigma=2.55e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  '

  100   1500 -1.313870131773152e+10 1.7e+00 1.69e+02  2e+02  2e+02 0:02.0
  106   1590 -3.054101641463115e+10 1.7e+00 2.64e+02  2e+02  3e+02 0:02.1
termination on tolfacupx=1000.0 (Mon Apr 13 14:48:49 2020)
final/bestever f-value = -2.808606e+10 -3.054102e+10
incumbent solution: [  348.79467437   691.45360596  -575.28477164   215.31078834
  1165.18451445  -260.24952187 -1285.54081134 -1278.96338006 ...]
std deviations: [269.13345079 264.97729656 251.40089973 249.38849663 260.489427
 255.58495738 263.08807464 257.63625436 ...]
Run : 1
(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimension 50 (seed=854383, Mon Apr 13 14:48:50 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     15 1.166785344391034e+04 1.0e+00 2.56e-01  3e-01  3e-01 0:00.0
    2     30 1.166704202081162e+04 1.0e+00 2.49e-01  2e-01  3e-01 0:00.0
    3     45 1.166697286015213e+04 1.1e+00 2.47e-01  2e-01  2e-01 0:00.1


...

  100   1500 -2.449080121898459e+09 1.7e+00 8.46e+01  8e+01  9e+01 0:01.9
  115   1725 -2.350598242675407e+10 1.7e+00 2.54e+02  2e+02  3e+02 0:02.2
termination on tolfacupx=1000.0 (Mon Apr 13 15:19:21 2020)
final/bestever f-value = -2.088978e+10 -2.350598e+10
incumbent solution: [-863.29527623 1319.94829998   34.84080922  498.03014527  844.65952851
  189.05205262  346.22153507 -278.68896061 ...]
std deviations: [250.05276708 256.24381282 241.77612483 246.17113611 243.52620087
 242.44909336 240.54985129 243.94440318 ...]
Run : 29
(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimension 50 (seed=895460, Mon Apr 13 15:19:21 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     15 1.166648580355900e+04 1.0e+00 2.55e-01  3e-01  3e-01 0:00.0
    2     30 1.166588205406059e+04 1.0e+00 2.46e-01  2e-01  2e-01 0:00.0
    3     45 1.166403985984729e+04 1.1e+00 2.43e-01  2e-01  2e-01 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [11668.43262211] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [11666.68468813] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1500 -1.045590254908776e+04 1.4e+00 3.02e-02  3e-02  3e-02 0:02.0
  200   3000 -1.136902468212598e+04 1.5e+00 1.77e-03  2e-03  2e-03 0:03.9
  300   4500 -1.137232189039848e+04 1.5e+00 8.96e-05  8e-05  8e-05 0:05.8
  400   6000 -1.137232774183210e+04 1.6e+00 6.41e-06  5e-06  6e-06 0:07.8
  500   7500 -1.137232776777503e+04 1.6e+00 3.33e-07  3e-07  3e-07 0:09.7
  600   9000 -1.137232776785019e+04 1.7e+00 2.02e-08  2e-08  2e-08 0:11.7
  700  10500 -1.137232776785041e+04 1.8e+00 2.34e-09  2e-09  2e-09 0:13.6
  800  12000 -1.137232776785041e+04 1.9e+00 1.26e-09  9e-10  1e-09 0:15.6
  900  13500 -1.137232776785041e+04 2.2e+00 1.12e-09  8e-10  1e-09 0:17.5
 1000  15000 -1.137232776785041e+04 2.4e+00 9.03e-10  6e-10  8e-10 0:19.5
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  16500 -1.137232776785041e+04 2.5e+00 5.82e-10  4e-10  5e-10 0:21.4
 1200  18000 -1.137232776785041e+04 2.8e+00 6.86e-10  5e-10  6e-10 0:23.3
 1300  19500 -1.137232776785041e+04 3.2e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [10895.81357622] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=10895.813576, sigma=2.54e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=10895.813576, sigma=2.54e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  '

  100   1500 -6.871905022149156e+09 1.6e+00 1.32e+02  1e+02  1e+02 0:01.9
  109   1635 -2.709849281346748e+10 1.6e+00 2.73e+02  3e+02  3e+02 0:02.1
termination on tolfacupx=1000.0 (Mon Apr 13 15:20:18 2020)
final/bestever f-value = -2.537804e+10 -2.709849e+10
incumbent solution: [-1064.98207905   -85.32497274   410.68843484  1092.23684945
   151.68899631   197.84242702  1674.01586874   334.67984958 ...]
std deviations: [260.80570447 263.83705635 263.68036407 262.0492538  268.68504596
 269.20489083 265.97747261 267.69626202 ...]


In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('F15_100_X_Values.csv')
Krig_Fun = np.zeros(30)
SVM_Fun = np.zeros(30)
KNN_Fun = np.zeros(30)
ELN_Fun = np.zeros(30)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F15(X_Values.iloc[i,:50])
    SVM_Fun [i] = F15(X_Values.iloc[i,50:100])
    ELN_Fun [i] = F15(X_Values.iloc[i,100:150])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([2307.70849554]), count=array([1])) 3.7612045885452633e-06
SVM
ModeResult(mode=array([1916.80781963]), count=array([1])) 145.5192150507457
ELN
ModeResult(mode=array([4.59975301e+39]), count=array([1])) 2.3496366787902294e+66
